In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 'inference/get-started',
 '/spaces',
 '/models',
 '/moonshotai/Kimi-K2-Instruct',
 '/Qwen/Qwen3-235B-A22B-Instruct-2507',
 '/mistralai/Voxtral-Mini-3B-2507',
 '/mistralai/Voxtral-Small-24B-2507',
 '/nvidia/canary-qwen-2.5b',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces/akhaliq/anycoder',
 '/spaces/multimodalart/wan2-1-fast',
 '/spaces/NihalGazi/Text-To-Speech-Unlimited',
 '/spaces',
 '/datasets/NousResearch/Hermes-3-Dataset',
 '/datasets/common-pile/caselaw_access_project',
 '/datasets/microsoft/rStar-Coder',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/snorkelai/agent-finance-reasoning',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Get started with Inference in seconds 🚀
Reachy Mini: The Open Robot for AI Builders
Welcome Cohere on the Hub 🔥
The AI community building the future.
The platform where

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nGet started with Inference in seconds 🚀\nReachy Mini: The Open Robot for AI Builders\nWelcome Cohere on the Hub 🔥\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmoonshotai/Kimi-K2-Instruct\nUpdated\nabout 10 hours ago\n•\n195k\n•\n1.69k\nQwen/Qwen3-235B-A22B-Instruct-2507\nUpdated\n1 day ago\n•\n1.32k\n•\n319\nmistralai/Voxtral-Mini-3B-2507\nUpdated\n4 days ago\n•\n46.4k\n•\n399\nmistralai/Voxtral-Small-24B-2507\nUpdated\n4 days 

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'forum page', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}]}


# Hugging Face: Building the Future of AI

## About Us
Hugging Face is a vibrant AI community dedicated to shaping the future through collaborative machine learning. We focus on creating a platform where developers, researchers, and enthusiasts come together to share knowledge, models, datasets, and applications.

## Our Platform Features
- **Models**: Browse over 1 million models across various AI modalities (text, image, video, audio, and 3D).
- **Datasets**: Access and share more than 250,000 datasets tailored for a variety of ML tasks.
- **Spaces**: Deploy applications with ease. From image editing to text-to-speech, our platform supports diverse AI applications.
- **Community**: Engage with peers and contribute to collective projects. Be a part of the leading open-source movement for AI.

## Who We Serve
With over 50,000 organizations using our platform, some of our esteemed clients include:
- **Meta**
- **Amazon**
- **Google**
- **Microsoft**

These companies leverage our advanced tools to accelerate their machine learning projects and innovations.

## Company Culture
At Hugging Face, we champion an inclusive and collaborative work environment. Our culture is defined by a commitment to open-source principles, innovation, and community engagement. We empower our team members to push the boundaries of AI and support their personal growth through continuous learning.

### Why Join Us?
- **Diverse Opportunities**: Work on transformative projects and cutting-edge technology.
- **Growth-Focused**: We invest in your development through ongoing training and mentorship.
- **Community Oriented**: Be part of a lively community committed to advancing AI while making meaningful contributions.

### Current Openings
Explore exciting career opportunities that match your skills and passions on our [Jobs Page](https://huggingface.co/jobs).

## Get Involved
Join us in shaping the future of AI by signing up and contributing to our platform. Whether you're a developer, researcher, or AI enthusiast, there’s a place for you at Hugging Face.

### Connect with Us
- [Website](https://huggingface.co)
- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://linkedin.com/company/huggingface)

---

Join Hugging Face today and be part of a community that's building the future of artificial intelligence, one model at a time!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}]}


# Hugging Face Brochure

## About Us
**Hugging Face** is at the forefront of the artificial intelligence community, dedicated to building the future of machine learning. Our platform enables collaboration among developers, researchers, and enterprises to create, share, and discover machine learning models, datasets, and applications. With a commitment to openness and community-driven growth, Hugging Face empowers innovators to accelerate their machine learning projects with state-of-the-art tools.

---

## Our Offerings

- **Models & Datasets**: Explore an extensive repository of over **1 million models** and **250,000 datasets** tailored for various applications.
- **Spaces**: Engage in collaborative work with over **10,700 applications** running on our platform.
- **Enterprise Solutions**: Enhance your team’s efficiency with enterprise-grade features, including dedicated support and enhanced security. Starting at **$20/user/month**.
- **Open Source Tools**: Build on the foundation of machine learning with our widely-used libraries including **Transformers**, **Diffusers**, and more.

---

## Who We Serve
More than **50,000 organizations** utilize Hugging Face, including industry giants like:
- Meta
- Amazon
- Google
- Microsoft
- Grammarly

Our diverse customer base ranges from startups to large enterprises working to harness AI for various applications.

---

## Company Culture
At Hugging Face, we believe in the power of community and open collaboration. Our culture fosters innovation where every team member's input is valued. We encourage employees to explore creativity, engage in continuous learning, and participate in shaping the future of AI. We pride ourselves on maintaining an inclusive environment that brings together talents from diverse backgrounds.

---

## Careers at Hugging Face
We are always on the lookout for passionate individuals who want to join our growing team. Whether you're a developer, data scientist, designer, or marketing professional, if you're eager to make a difference in the AI field, we want to hear from you!

Visit our **[Jobs page](https://huggingface.co/jobs)** to discover current opportunities and start your journey with us.

---

## Join the Community
Become a part of the vibrant Hugging Face community! Collaborate, share knowledge, and enhance your skills through our forums, social media, and collaborative coding initiatives.

- **GitHub**: contributing to open-source projects
- **Twitter**: follow our latest updates
- **LinkedIn**: connect with us professionally
- **Discord**: engage in discussions with fellow AI enthusiasts

---

## Get Started
Embark on your AI journey with Hugging Face today! Explore our offerings, sign up for our services, and start building innovative solutions that impact the future.

**[Join us!](https://huggingface.co)**

---

Embrace the potential of AI. Be part of the Hugging Face story and help shape the future of technology.

In [21]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://github.com/huggingface'}, {'type': 'company page', 'url': 'https://twitter.com/huggingface'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

## About Us
**Hugging Face** is at the forefront of the artificial intelligence community, where innovators collaborate to build the future of machine learning (ML). Our platform allows members to share, discover, and collaborate on models, datasets, and applications. With over **1 million models** and **250,000 datasets**, we empower AI developers and researchers to create transformative technologies.

---

## What We Offer
### Key Features
- **Models & Datasets**: Access a vast repository of models and datasets tailored for various machine learning tasks.
- **Spaces**: An interactive platform to showcase applications and projects, running over **10,700** live applications.
- **Enterprise Solutions**: Comprehensive services for businesses looking to deploy ML at scale with enterprise-grade security and support. Starting at **$20/user/month**.

### Open Source Tools
We provide cutting-edge open-source libraries such as:
- **Transformers**: Leading ML framework for PyTorch, TensorFlow, and JAX.
- **Diffusers**: Advanced diffusion models.
- **Tokenizers**: Fast and efficient tokenization for ML applications.

---

## Our Community
At Hugging Face, community is at the heart of everything we do. With over **50,000 organizations** using our platform, including giants like Google, Microsoft, and Amazon, we foster a collaborative environment. Our community-driven approach invites users to participate in model development, contribute to datasets, and exchange ideas freely.

---

## Company Culture
### Values
- **Inclusion**: We celebrate diversity and nurture an environment where everyone's contribution is valued.
- **Innovation**: Continuous learning and development are central to our ethos, inspiring our team to push the boundaries of AI.
- **Collaboration**: We believe that the best technologies emerge from collective efforts and shared knowledge.

### Careers at Hugging Face
If you're passionate about AI and want to join a vibrant community of thinkers and makers, explore our [careers page](#) for current opportunities. From engineering to product management and beyond, we're looking for individuals who are ready to tackle the challenges of the AI landscape.

---

## Get Involved
- **Explore** our platform and discover various models and datasets.
- **Join** the Hugging Face community on platforms such as [Discord](#), [Twitter](#), and [LinkedIn](#).
- **Contribute** to open-source projects that are shaping the future of AI.

---

### Contact Us
Join us in building tomorrow's AI! Visit our website at [Hugging Face](https://huggingface.co) for more details or connect with us directly.

--- 

*Empowering the machine learning community to unlock the full potential of AI.*

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>